### 🟢 Step 1 — Import libraries & define metric

In [1]:
# Imports + Metric
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
import joblib
# Define MdAPE — Median Absolute Percentage Error
def mdape(y_true, y_pred):
    return np.median(np.abs((y_true - y_pred) / y_true) * 100)

### 🟢 Step 2 — Load Training + New 2022 Data

In [2]:
# load base training and new incremental data
train = pd.read_csv("train.csv")
data_2022 = pd.read_csv("data_2022.csv")

print("Train:", train.shape)
print("2022 Data:", data_2022.shape)

Train: (46499, 14)
2022 Data: (7003, 14)


### 🟢 Step 3 — Combine Datasets

In [3]:
# combine historical + 2022 data
combined = pd.concat([train, data_2022], ignore_index=True)

print("Combined dataset:", combined.shape)

Combined dataset: (53502, 14)


### 🟢 Step 4 — Encode & Build Features (SAME as baseline model)

In [4]:
# Encode property_type
enc = LabelEncoder()
combined["property_type_enc"] = enc.fit_transform(combined["property_type"])

baseline_features = [
    "latitude","longitude","sqft","beds","baths",
    "property_type_enc","days_since_2014"
]

target_col = "trans_log_price"

X_train_new = combined[baseline_features]
y_train_new = combined[target_col]

print("Training matrix:", X_train_new.shape)


Training matrix: (53502, 7)


### 🟢 Step 5 — Train Refreshed Model

In [5]:
# retrain baseline model on expanded dataset
retrain_model = XGBRegressor(
    max_depth=6, n_estimators=300, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8, random_state=42
)

retrain_model.fit(X_train_new, y_train_new)

print("Retraining completed.")

Retraining completed.


### 🟢 Step 6 — Save Model + Encoder

In [6]:
# save model + encoder for reuse
joblib.dump(retrain_model, "beekin_model_2022.pkl")
joblib.dump(enc, "property_type_encoder.pkl")

print("Saved: beekin_model_2022.pkl")
print("Saved: property_type_encoder.pkl")


Saved: beekin_model_2022.pkl
Saved: property_type_encoder.pkl
